# 7. Spatial dependence models

**Warning! This chapter is realized in R**

In this chapter we create spatial dependence econometric models to answer our research hypothesis. We believe based on spatial autocorrelation tests (performed in previous chapters) that spatial autocorrelation exists in the analyzed phenomenon. We create models for Warsaw and Cracow separately. We utilize here 1km2 grid!

Generally, we specific following functional form of the models:
$inpost_i = \alpha * \text{logistic_companies}_i + \beta * \text{demographic_variables}_i + \gamma * \text{point_of_interest_variables}_i + \theta * \text{spatial_terms}_i + \epsilon$, 

where:

* logistic_companies cover: DHL, DPD, Fedex, Poczta Polska, Ruch and UPS
* demographic_variables cover: total density, density age 0-14, density age 15-64, density age 65-...,  total mail density, total female density, female density age 0-14, female density age 15-64, female density age 65-..., male density age 0-14, male density age 15-64, male density age 65-..., feminization ratio
* point_of_interest_variables cover: buildings, shops, parks, forests, schools, railways, cycleways, parkings, crossings, bus stops

Obviously, a significant proportion of these variables will be removed from the final model due to high co-linearity or statistical insignificance, however, based on the literature and our intuition, we assume that at least 1-2 variables from each category will ultimately be significant. 

Our expectations for the results are as follows (taking into account the research hypotheses). Based on the visual analysis of spatial data carried out in the previous chapters of this work, it seems to us that InPost has deployed parcel pick-up points in accordance with the competition. Therefore, we expect the variables in the logistic_companies category to be significant and have a positive sign. In addition, it seems to us that the control variables (groups: demographic_variables, point_of_interest_variables) should also be important in the models and their sign should follow the logic: the more shops in the grid, the more parcel machines, the greater the afforestation, the fewer parcel machines, the more bus stops the more parcel machines, the higher the population density, the more parcel machines etc. In addition, we believe that extending the OLS model with spatial components is a good idea and models that take into account spillover effects will be econometrically better. We apply these expectations to both Warsaw and Cracow.

We adopted the following modeling strategy due to the complexity of the problem:

1. OLS model estimation to establish an initial functional form that is free of collinearity (VIF statistic) of the variables along with model diagnostics.
2. Estimation of the final OLS model using Stepwise Regression procedure (both directions).
3. Estimation of spatial models with 3, 2, 1 spatial components (using obtimate functional form from point 2).
4. Comparing the statistical properties of the models from step 3 and selecting the target model. We look for model with significant spatial components, highest number of significant variables and minimum AIC (we also take into account BIC).
5. For the obtimate model in terms of spatial components, we test other combinations of variables using the general to specific (LSE) procedure. 
6. Interpret the results of the model from point 5 


We covered all possible options to improve the models:
-	Changing spatial components (done - all possible components tried)
-	Changing the spatial weights matrix (done - QUEEN and ROOK tested)
-	Changing variables in the model (done - general to specific approach to make a choice for OLS and then infer based on it)

We set 10% significance level!

## Import dependencies

In [72]:
library(rgdal)
library(spdep)
library(tidyverse)
library(car)
library(lmtest)
library(arules)
library(spatialreg)
library(texreg)
library(stargazer)

## Models for Warsaw

### Data loading, spatial transformations and additional feature engineering

In [2]:
df <- readOGR("../datasets/preprocessed_data/df_warszawa.shp")
df_data <- read.csv("../datasets/preprocessed_data/df_warszawa.csv")
df <- spTransform(df, CRS("+proj=longlat +ellps=GRS80 +no_defs"))
df.cont.nb <- poly2nb(as(df, "SpatialPolygons"), queen=TRUE) #QUEEN but ROOK was tried!
df.cont.listw <- nb2listw(df.cont.nb, style="W")

OGR data source with driver: ESRI Shapefile 
Source: "D:\spatial_econometric_project\datasets\preprocessed_data\df_warszawa.shp", layer: "df_warszawa"
with 601 features
It has 31 fields
Integer64 fields read as strings:  grid_index 


In [73]:
cat(colnames(df_data))

grid_index geometry dhl dpd fedex inpost poczta ruch ups tot tot_0_14 tot_15_64 tot_65__ tot_male tot_fem male_0_14 male_15_64 male_65__ fem_0_14 fem_15_64 fem_65__ fem_ratio buildings shops parks forests schools railways cycleways parkings crossings bus_stops

We investigate again correlation between logistic companies. We can clearly see that Poczta Polska is highly collinear with other variables so we will remove it from the further analysis. The rest variables seems to be ok.

In [3]:
stats::cor(df_data %>% select('dhl', 'dpd', 'fedex', 'inpost', 'poczta', 'ruch', 'ups'))

,dhl,dpd,fedex,inpost,poczta,ruch,ups
dhl,1.0000000,0.6277199,0.18363265,0.7380562,0.9238711,0.6356515,0.53454845
dpd,0.6277199,1.0000000,0.13532748,0.5657193,0.6875670,0.5941321,0.43918066
fedex,0.1836326,0.1353275,1.00000000,0.2048367,0.1740457,0.1312876,0.08168199
inpost,0.7380562,0.5657193,0.20483665,1.0000000,0.7411384,0.6494003,0.53922648
poczta,0.9238711,0.6875670,0.17404573,0.7411384,1.0000000,0.7454525,0.57380202
ruch,0.6356515,0.5941321,0.13128761,0.6494003,0.7454525,1.0000000,0.53308523
ups,0.5345484,0.4391807,0.08168199,0.5392265,0.5738020,0.5330852,1.00000000


Potential enhancement of the features.

In [4]:
df$tot_0_14 <- df$tot_0_14/df$tot
df$tot_15_64 <- df$tot_15_64/df$tot
df$tot_65__ <- df$tot_65__/df$tot

df$male_0_14 <- df$male_0_14/df$tot_male 
df$male_15_64 <- df$male_15_64/df$tot_male 
df$male_65__ <- df$male_65__/df$tot_male 

df$fem_0_14 <- df$fem_0_14/df$tot_fem 
df$fem_15_64 <- df$fem_15_64/df$tot_fem 
df$fem_65__ <- df$fem_65__/df$tot_fem 

### OLS models

Based on many tries we decided that below formula is the most appropriate general formula for the given problem. We removed Poczta Polska from logistic companies, we left total density and feminization ratio in case of demographic data and finally we leave all data regarding points of interest.

In [5]:
formula = inpost ~ dhl + dpd + fedex + ruch + ups +
            log(tot + 1) + fem_ratio + 
            buildings + shops + parks + forests + schools + 
            railways + cycleways + parkings + crossings + bus_stops

We estimate OLS model and we perform its diagnostics. We see that nearly 50% of the variables are significant (2 or more per each category of data). We do not have now high collinearity. However this model did not pass Ramsey and Breusch–Pagan test so the functional form is wrong and we have got heteroscedasticity in the errors. It occurred that at the level of 10% significance our residuals are spatially dependent!

In [6]:
ols <- lm(formula, data=df)
summary(ols)
car::vif(ols)
bptest(ols) 
resettest(ols, power=2, type="regressor")
lm.morantest(ols, df.cont.listw) 


Call:
lm(formula = formula, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.6926 -0.5928 -0.1367  0.3983  5.4677 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -1.095e-01  1.768e-01  -0.619 0.536066    
dhl           2.425e-01  3.221e-02   7.529 1.95e-13 ***
dpd           4.465e-02  7.660e-02   0.583 0.560192    
fedex         4.634e-01  3.063e-01   1.513 0.130902    
ruch          3.813e-01  8.948e-02   4.262 2.37e-05 ***
ups           3.945e-01  1.120e-01   3.521 0.000463 ***
log(tot + 1)  1.909e-01  3.953e-02   4.830 1.75e-06 ***
fem_ratio    -4.774e-03  2.312e-03  -2.065 0.039352 *  
buildings    -5.622e-05  9.314e-04  -0.060 0.951885    
shops        -3.495e-03  9.629e-02  -0.036 0.971055    
parks        -5.541e-02  1.443e-02  -3.839 0.000137 ***
forests      -6.320e-03  5.571e-03  -1.134 0.257150    
schools       7.159e-02  1.838e-02   3.895 0.000110 ***
railways      4.179e-03  3.056e-03   1.368 0.171990    
cycleways  

dhl          dpd        fedex         ruch          ups log(tot + 1) 
    3.430740     1.959789     1.069884     2.614402     1.661937     3.756089 
   fem_ratio    buildings        shops        parks      forests      schools 
    2.227054     1.140331     1.765262     1.431213     1.164145     3.224955 
    railways    cycleways     parkings    crossings    bus_stops 
    1.243783     1.804344     3.549422     4.525842     1.036397


	studentized Breusch-Pagan test

data:  ols
BP = 141.3, df = 17, p-value < 2.2e-16



	RESET test

data:  ols
RESET = 5.2474, df1 = 17, df2 = 566, p-value = 7.66e-11



	Global Moran I for regression residuals

data:  
model: lm(formula = formula, data = df)
weights: df.cont.listw

Moran I statistic standard deviate = 1.6831, p-value = 0.04618
alternative hypothesis: greater
sample estimates:
Observed Moran I      Expectation         Variance 
    0.0291485488    -0.0063435194     0.0004446764 


Then we applied Stepwise Regression procedure to obtain the most significant model starting from the function form defined above.

In [7]:
intercept_only <- lm(inpost~1, data=df)
all <- lm(formula, data=df)
both <- step(intercept_only, direction='both', scope=formula(all), trace=0)
both


Call:
lm(formula = inpost ~ dhl + schools + log(tot + 1) + ruch + parks + 
    ups + crossings + fem_ratio + fedex, data = df)

Coefficients:
 (Intercept)           dhl       schools  log(tot + 1)          ruch  
   -0.144470      0.258789      0.075243      0.192578      0.379413  
       parks           ups     crossings     fem_ratio         fedex  
   -0.058957      0.379756      0.004715     -0.004973      0.482499  


Based on Stepwise Regression we obtained final formula of the model and we are able to run our final OLS with diagnostic part. We see that nearly all variables in the model are significant (of course due to the heteroscedasticity we use biased error variance estimator - we omitted application of the robust matrix due to lack of time). There is no collinearity in the model, but again we did not pass Ramsey and B-P tests. What is more residuals from the model are still spatially dependent. In case of  interpretation we can see that: logistic companies have positive sign as expected; the greater the total density the more InPost pick-up points are stated; fem_ratio results is interesting because its sign is negative (more woman less pick-up points, quite spurious?!), in addition, the negative relationship between parks and pickup points is interesting but maybe intuitive.

In [8]:
formula = inpost ~ dhl + schools + log(tot + 1) + ruch + parks + 
    ups + crossings + fem_ratio + fedex
ols <- lm(formula, data=df)
summary(ols)
cat("AIC", AIC(ols))
car::vif(ols)
bptest(ols) 
resettest(ols, power=2, type="regressor")
lm.morantest(ols, df.cont.listw) 


Call:
lm(formula = formula, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.9841 -0.6204 -0.1485  0.3748  5.7877 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.144470   0.171182  -0.844 0.399034    
dhl           0.258789   0.028132   9.199  < 2e-16 ***
schools       0.075243   0.015650   4.808 1.94e-06 ***
log(tot + 1)  0.192578   0.039205   4.912 1.17e-06 ***
ruch          0.379413   0.081565   4.652 4.07e-06 ***
parks        -0.058957   0.013272  -4.442 1.06e-05 ***
ups           0.379756   0.108958   3.485 0.000528 ***
crossings     0.004715   0.002238   2.106 0.035612 *  
fem_ratio    -0.004973   0.002298  -2.164 0.030876 *  
fedex         0.482499   0.303603   1.589 0.112541    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.303 on 591 degrees of freedom
Multiple R-squared:  0.683,	Adjusted R-squared:  0.6782 
F-statistic: 141.5 on 9 and 591 DF,  p-value: < 2.2e-16


AIC 2035.98

dhl      schools log(tot + 1)         ruch        parks          ups 
    2.630252     2.349016     3.712366     2.182872     1.216059     1.579691 
   crossings    fem_ratio        fedex 
    3.563303     2.212064     1.055896


	studentized Breusch-Pagan test

data:  ols
BP = 133.08, df = 9, p-value < 2.2e-16



	RESET test

data:  ols
RESET = 6.8392, df1 = 9, df2 = 582, p-value = 2.223e-09



	Global Moran I for regression residuals

data:  
model: lm(formula = formula, data = df)
weights: df.cont.listw

Moran I statistic standard deviate = 1.4688, p-value = 0.07094
alternative hypothesis: greater
sample estimates:
Observed Moran I      Expectation         Variance 
    0.0268763136    -0.0042751978     0.0004497972 


### Spatial models with 3, 2, or 1 spatial components

Now we moved to the estimation of the spatial models. In the first step we estimate all possible models due to spatial terms and then we analyze which model is reasonable (based on number of significant variables, significant spatial terms and AIC+BIC).

In [9]:
GNS <- sacsarlm(formula, data=df, listw=df.cont.listw, type="sacmixed", method="LU")
summary(GNS)


Call:sacsarlm(formula = formula, data = df, listw = df.cont.listw, 
    type = "sacmixed", method = "LU")

Residuals:
     Min       1Q   Median       3Q      Max 
-4.65144 -0.59387 -0.10078  0.39765  5.29613 

Type: sacmixed 
Coefficients: (numerical Hessian approximate standard errors) 
                   Estimate Std. Error z value  Pr(>|z|)
(Intercept)      -0.3096406  0.3759817 -0.8236  0.410194
dhl               0.2228032  0.0321699  6.9258 4.335e-12
schools           0.0805577  0.0174675  4.6119 3.991e-06
log(tot + 1)      0.1356923  0.0439068  3.0905  0.001998
ruch              0.4126336  0.0829722  4.9732 6.587e-07
parks            -0.0388209  0.0159319 -2.4367  0.014823
ups               0.3522209  0.1076515  3.2719  0.001068
crossings         0.0067908  0.0026043  2.6076  0.009119
fem_ratio        -0.0038325  0.0023313 -1.6440  0.100181
fedex             0.6562935  0.3025839  2.1690  0.030085
lag.dhl           0.1202994  0.1140963  1.0544  0.291715
lag.schools      -0.04096

In [10]:
SAC <- sacsarlm(formula, data=df, listw=df.cont.listw)
summary(SAC)


Call:sacsarlm(formula = formula, data = df, listw = df.cont.listw)

Residuals:
     Min       1Q   Median       3Q      Max 
-4.77462 -0.60148 -0.15324  0.41148  5.76569 

Type: sac 
Coefficients: (asymptotic standard errors) 
               Estimate Std. Error z value  Pr(>|z|)
(Intercept)  -0.1721734  0.1729029 -0.9958 0.3193566
dhl           0.2477643  0.0287298  8.6239 < 2.2e-16
schools       0.0723287  0.0158275  4.5698 4.881e-06
log(tot + 1)  0.1809632  0.0391950  4.6170 3.893e-06
ruch          0.3864329  0.0807050  4.7882 1.683e-06
parks        -0.0596580  0.0133850 -4.4571 8.308e-06
ups           0.3753236  0.1079086  3.4782 0.0005049
crossings     0.0041734  0.0023173  1.8010 0.0717103
fem_ratio    -0.0047245  0.0022738 -2.0778 0.0377256
fedex         0.4830808  0.2999343  1.6106 0.1072621

Rho: 0.074328
Asymptotic standard error: 0.064193
    z-value: 1.1579, p-value: 0.24691
Lambda: 0.024735
Asymptotic standard error: 0.10089
    z-value: 0.24518, p-value: 0.80632

LR test 

In [11]:
SDM <- errorsarlm(formula, data=df, listw=df.cont.listw, etype="mixed")
summary(SDM)


Call:errorsarlm(formula = formula, data = df, listw = df.cont.listw, 
    etype = "mixed")

Residuals:
      Min        1Q    Median        3Q       Max 
-4.757531 -0.597547 -0.095155  0.389789  5.263433 

Type: error 
Coefficients: (asymptotic standard errors) 
                   Estimate Std. Error z value  Pr(>|z|)
(Intercept)      -0.4234932  0.4373072 -0.9684  0.332839
dhl               0.2223463  0.0310939  7.1508 8.629e-13
schools           0.0800192  0.0169938  4.7087 2.493e-06
log(tot + 1)      0.1461070  0.0423166  3.4527  0.000555
ruch              0.3927065  0.0797844  4.9221 8.562e-07
parks            -0.0423316  0.0153067 -2.7656  0.005683
ups               0.3401743  0.1063629  3.1982  0.001383
crossings         0.0068412  0.0025220  2.7126  0.006676
fem_ratio        -0.0043878  0.0022628 -1.9391  0.052495
fedex             0.6458184  0.3077289  2.0987  0.035847
lag.dhl           0.2297732  0.0727347  3.1591  0.001583
lag.schools      -0.0208795  0.0358180 -0.5829  0.55

In [12]:
SDEM <- errorsarlm(formula, data=df, listw=df.cont.listw, etype="emixed")
summary(SDEM)


Call:errorsarlm(formula = formula, data = df, listw = df.cont.listw, 
    etype = "emixed")

Residuals:
      Min        1Q    Median        3Q       Max 
-4.757531 -0.597547 -0.095155  0.389789  5.263433 

Type: error 
Coefficients: (asymptotic standard errors) 
                   Estimate Std. Error z value  Pr(>|z|)
(Intercept)      -0.4234932  0.4373072 -0.9684  0.332839
dhl               0.2223463  0.0310939  7.1508 8.629e-13
schools           0.0800192  0.0169938  4.7087 2.493e-06
log(tot + 1)      0.1461070  0.0423166  3.4527  0.000555
ruch              0.3927065  0.0797844  4.9221 8.562e-07
parks            -0.0423316  0.0153067 -2.7656  0.005683
ups               0.3401743  0.1063629  3.1982  0.001383
crossings         0.0068412  0.0025220  2.7126  0.006676
fem_ratio        -0.0043878  0.0022628 -1.9391  0.052495
fedex             0.6458184  0.3077289  2.0987  0.035847
lag.dhl           0.2297732  0.0727347  3.1591  0.001583
lag.schools      -0.0208795  0.0358180 -0.5829  0.5

In [13]:
SEM <- errorsarlm(formula, data=df, listw=df.cont.listw)
summary(SEM)


Call:errorsarlm(formula = formula, data = df, listw = df.cont.listw)

Residuals:
     Min       1Q   Median       3Q      Max 
-4.81062 -0.60202 -0.18188  0.39096  5.84370 

Type: error 
Coefficients: (asymptotic standard errors) 
               Estimate Std. Error z value  Pr(>|z|)
(Intercept)  -0.1301997  0.1734180 -0.7508  0.452782
dhl           0.2530099  0.0283798  8.9151 < 2.2e-16
schools       0.0758115  0.0158034  4.7972 1.609e-06
log(tot + 1)  0.1823923  0.0394561  4.6227 3.788e-06
ruch          0.3885116  0.0808737  4.8039 1.556e-06
parks        -0.0560362  0.0134822 -4.1563 3.234e-05
ups           0.3819184  0.1078447  3.5414  0.000398
crossings     0.0050275  0.0022632  2.2214  0.026325
fem_ratio    -0.0046251  0.0022768 -2.0314  0.042218
fedex         0.4883269  0.2992225  1.6320  0.102682

Lambda: 0.097709, LR test value: 1.5928, p-value: 0.20693
Asymptotic standard error: 0.074295
    z-value: 1.3152, p-value: 0.18846
Wald statistic: 1.7296, p-value: 0.18846

Log likeli

In [20]:
SAR <- lagsarlm(formula, data=df, listw=df.cont.listw)
summary(SAR)


Call:lagsarlm(formula = formula, data = df, listw = df.cont.listw)

Residuals:
     Min       1Q   Median       3Q      Max 
-4.79287 -0.60136 -0.15215  0.41224  5.74634 

Type: lag 
Coefficients: (asymptotic standard errors) 
               Estimate Std. Error z value  Pr(>|z|)
(Intercept)  -0.1786277  0.1706989 -1.0464 0.2953537
dhl           0.2478015  0.0282846  8.7610 < 2.2e-16
schools       0.0717215  0.0156378  4.5864 4.509e-06
log(tot + 1)  0.1821701  0.0389206  4.6806 2.861e-06
ruch          0.3845874  0.0806541  4.7684 1.857e-06
parks        -0.0605122  0.0132188 -4.5778 4.700e-06
ups           0.3736604  0.1078399  3.4650 0.0005303
crossings     0.0040075  0.0022673  1.7675 0.0771456
fem_ratio    -0.0047848  0.0022728 -2.1052 0.0352740
fedex         0.4814659  0.3002205  1.6037 0.1087785

Rho: 0.084055, LR test value: 2.8325, p-value: 0.092373
Asymptotic standard error: 0.048482
    z-value: 1.7337, p-value: 0.082964
Wald statistic: 3.0059, p-value: 0.082964

Log likelihood

In [15]:
SLX <- lmSLX(formula, data=df, listw=df.cont.listw)
summary(SLX)


Call:
lm(formula = formula(paste("y ~ ", paste(colnames(x)[-1], collapse = "+"))), 
    data = as.data.frame(x), weights = weights)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.8073 -0.5970 -0.0874  0.3925  5.2600 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)      -0.419836   0.434336  -0.967 0.334137    
dhl               0.223368   0.031732   7.039 5.47e-12 ***
schools           0.080110   0.017349   4.618 4.78e-06 ***
log.tot...1.      0.146202   0.043149   3.388 0.000751 ***
ruch              0.392732   0.081145   4.840 1.67e-06 ***
parks            -0.041995   0.015633  -2.686 0.007431 ** 
ups               0.340498   0.108101   3.150 0.001717 ** 
crossings         0.006810   0.002574   2.645 0.008378 ** 
fem_ratio        -0.004387   0.002300  -1.907 0.056960 .  
fedex             0.647918   0.311972   2.077 0.038254 *  
lag.dhl           0.226797   0.073165   3.100 0.002030 ** 
lag.schools      -0.021032   0.035888  -0.586 0.55

Based on the above outputs we can clearly state that the best spatial model is **SAR (Spatial Lag Model)** - significant spatial terms and many significant variables.

Then we compare models BIC and AIC. We can clearly see that the best model in comparison to the OLS is SAR, however its BIC is greater that OLS BIC. In case of AIC which is more important the best model is SLX. However based on its output we can claim that SAR is still better in case of econometric inference.

In [30]:
cat("ols",BIC(ols) ,'\n')
cat("GNS",BIC(GNS),'\n')
cat("SAC",BIC(SAC),'\n')
cat("SDM",BIC(SDM),'\n')
cat("SDEM",BIC(SDEM),'\n')
cat("SAR",BIC(SAR),'\n')
cat("SLX",BIC(SLX),'\n')
cat("SEM",BIC(SEM),'\n')
cat('\n')
cat("ols",AIC(ols) ,'\n')
cat("GNS",AIC(GNS),'\n')
cat("SAC",AIC(SAC),'\n')
cat("SDM",AIC(SDM),'\n')
cat("SDEM",AIC(SDEM),'\n')
cat("SAR",AIC(SAR),'\n')
cat("SLX",AIC(SLX),'\n')
cat("SEM",AIC(SEM),'\n')

ols 2084.365 
GNS 2127.256 
SAC 2094.27 
SDM 2121.4 
SDEM 2121.4 
SAR 2087.931 
SLX 2115.285 
SEM 2089.17 

ols 2035.98 
GNS 2030.487 
SAC 2037.088 
SDM 2029.029 
SDEM 2029.029 
SAR 2035.147 
SLX 2027.313 
SEM 2036.387 


Now we wanted to find better functional form for the SAR model in the general to specific procedure (LSA) done by hand, however we were not able to improve this model (our previous functional form was the best one). Residuals in the model are not autocorrelated.

In [27]:
formula_max = inpost ~ dhl + dpd + fedex + ruch + ups +
            log(tot + 1) + fem_ratio + 
            buildings + shops + parks + forests + schools + 
            railways + cycleways + parkings + crossings + bus_stops

formula_gen_to_specifc = inpost ~ dhl + fedex + ruch + ups +
            log(tot + 1) + fem_ratio + 
            parks + schools + 
            crossings

SAR_g2s<- lagsarlm(formula_gen_to_specifc, data=df, listw=df.cont.listw)
summary(SAR_g2s) #the same as previously!


Call:
lagsarlm(formula = formula_gen_to_specifc, data = df, listw = df.cont.listw)

Residuals:
     Min       1Q   Median       3Q      Max 
-4.79287 -0.60136 -0.15215  0.41224  5.74634 

Type: lag 
Coefficients: (asymptotic standard errors) 
               Estimate Std. Error z value  Pr(>|z|)
(Intercept)  -0.1786277  0.1706989 -1.0464 0.2953537
dhl           0.2478015  0.0282846  8.7610 < 2.2e-16
fedex         0.4814659  0.3002205  1.6037 0.1087785
ruch          0.3845874  0.0806541  4.7684 1.857e-06
ups           0.3736604  0.1078399  3.4650 0.0005303
log(tot + 1)  0.1821701  0.0389206  4.6806 2.861e-06
fem_ratio    -0.0047848  0.0022728 -2.1052 0.0352740
parks        -0.0605122  0.0132188 -4.5778 4.700e-06
schools       0.0717215  0.0156378  4.5864 4.509e-06
crossings     0.0040075  0.0022673  1.7675 0.0771456

Rho: 0.084055, LR test value: 2.8325, p-value: 0.092373
Asymptotic standard error: 0.048482
    z-value: 1.7337, p-value: 0.082964
Wald statistic: 3.0059, p-value: 0.082964

### Direct and indirect impacts

Now we calculate direct and indirect impacts of the model. And we finally analyze its output.

Let's start with logistic companies variables. We see that either in case of direct and indirect impacts all parameters for logistic companies (apart indirect Fedex) are significant (at least around 10%). Internalization effects are greater than spill-over, but still spill-over effects are significant. In case of demographic variables we claim that total population density is important variable for the model in case of direct and indirect impact (has positive sign). Feminization ratio in case of direct impact is also significant and has negative sign (it is quite non intuitive). In case of point of interest data we see that number of schools, number of parks and number of crossings are significant in the model (crossings spillover effect is not relevant) with respectively positive, negative and positive sign. We claim that it might be intuitive.

In [31]:
# distribution of total impact 
# vector of traces of powers of a spatial weights matrix
# converting censored listw to CsparseMatrix form
W.c <- as(as_dgRMatrix_listw(df.cont.listw), "CsparseMatrix") 
# the default values for the number of powers is 30
trMat<-trW(W.c, type="mult") 

SAR_imp<-impacts(SAR, tr=trMat, R=200)
summary(SAR_imp, zstats=TRUE, short=TRUE)

Impact measures (lag, trace):
                   Direct      Indirect        Total
dhl           0.248049502  0.0224925064  0.270542008
schools       0.071793286  0.0065100350  0.078303321
log(tot + 1)  0.182352374  0.0165352557  0.198887629
ruch          0.384972296  0.0349083218  0.419880618
parks        -0.060572761 -0.0054925860 -0.066065347
ups           0.374034348  0.0339164961  0.407950844
crossings     0.004011499  0.0003637527  0.004375252
fem_ratio    -0.004789587 -0.0004343077 -0.005223894
fedex         0.481947745  0.0437018122  0.525649557
Simulation results ( variance matrix):
Simulated standard errors
                  Direct     Indirect       Total
dhl          0.024577792 0.0129272070 0.028335501
schools      0.015146244 0.0039862622 0.016354111
log(tot + 1) 0.040420150 0.0102503478 0.044770293
ruch         0.088578780 0.0225102250 0.099527465
parks        0.012891407 0.0034663042 0.014522086
ups          0.113698964 0.0211674881 0.123919020
crossings    0.002284907 

## Models for Cracow

For the Cracow we applied the same procedure as for Warsaw, so we will omit here additional comments and focus only on the results.

### Data loading, spatial transformations and additional feature engineering

In [36]:
df <- readOGR("../datasets/preprocessed_data/df_krakow.shp")
df_data <- read.csv("../datasets/preprocessed_data/df_krakow.csv")
df <- spTransform(df, CRS("+proj=longlat +ellps=GRS80 +no_defs"))
df.cont.nb <- poly2nb(as(df, "SpatialPolygons"), queen=TRUE) #QUEEN but ROOK was tried!
df.cont.listw <- nb2listw(df.cont.nb, style="W")

OGR data source with driver: ESRI Shapefile 
Source: "D:\spatial_econometric_project\datasets\preprocessed_data\df_krakow.shp", layer: "df_krakow"
with 396 features
It has 31 fields
Integer64 fields read as strings:  grid_index 


In [37]:
cat(colnames(df_data))

grid_index geometry dhl dpd fedex inpost poczta ruch ups tot tot_0_14 tot_15_64 tot_65__ tot_male tot_fem male_0_14 male_15_64 male_65__ fem_0_14 fem_15_64 fem_65__ fem_ratio buildings shops parks forests schools railways cycleways parkings crossings bus_stops

In [38]:
stats::cor(df_data %>% select('dhl', 'dpd', 'fedex', 'inpost', 'poczta', 'ruch', 'ups'))

,dhl,dpd,fedex,inpost,poczta,ruch,ups
dhl,1.0000000,0.6382852,0.36539944,0.6853909,0.9543118,0.7208735,0.59141000
dpd,0.6382852,1.0000000,0.15220265,0.5960094,0.6589668,0.5101946,0.48551713
fedex,0.3653994,0.1522026,1.00000000,0.2529419,0.2965195,0.2026232,0.06370109
inpost,0.6853909,0.5960094,0.25294189,1.0000000,0.6777870,0.5782590,0.51096506
poczta,0.9543118,0.6589668,0.29651949,0.6777870,1.0000000,0.7762247,0.63031294
ruch,0.7208735,0.5101946,0.20262317,0.5782590,0.7762247,1.0000000,0.59953072
ups,0.5914100,0.4855171,0.06370109,0.5109651,0.6303129,0.5995307,1.00000000


In [39]:
df$tot_0_14 <- df$tot_0_14/df$tot
df$tot_15_64 <- df$tot_15_64/df$tot
df$tot_65__ <- df$tot_65__/df$tot

df$male_0_14 <- df$male_0_14/df$tot_male 
df$male_15_64 <- df$male_15_64/df$tot_male 
df$male_65__ <- df$male_65__/df$tot_male 

df$fem_0_14 <- df$fem_0_14/df$tot_fem 
df$fem_15_64 <- df$fem_15_64/df$tot_fem 
df$fem_65__ <- df$fem_65__/df$tot_fem 

### OLS models

In [40]:
formula = inpost ~ dhl + dpd + fedex + ruch + ups +
            log(tot + 1) + fem_ratio + 
            buildings + shops + parks + forests + schools + 
            railways + cycleways + parkings + crossings + bus_stops

In [41]:
ols <- lm(formula, data=df)
summary(ols)
car::vif(ols)
bptest(ols) 
resettest(ols, power=2, type="regressor")
lm.morantest(ols, df.cont.listw) 


Call:
lm(formula = formula, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.5947 -0.3813 -0.1155  0.2777  4.5829 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -2.488e-01  2.005e-01  -1.241 0.215269    
dhl           1.396e-01  5.011e-02   2.786 0.005611 ** 
dpd           2.882e-01  1.181e-01   2.441 0.015086 *  
fedex         3.097e-01  2.544e-01   1.217 0.224222    
ruch          1.630e-01  1.278e-01   1.275 0.202973    
ups           3.471e-01  1.401e-01   2.478 0.013649 *  
log(tot + 1)  1.759e-01  3.619e-02   4.860 1.72e-06 ***
fem_ratio    -4.458e-03  2.012e-03  -2.216 0.027299 *  
buildings     1.653e-04  1.429e-03   0.116 0.907966    
shops        -2.642e-02  4.412e-02  -0.599 0.549580    
parks        -1.900e-01  4.962e-02  -3.828 0.000151 ***
forests      -1.475e-03  4.636e-03  -0.318 0.750610    
schools       1.162e-01  2.165e-02   5.369 1.38e-07 ***
railways      3.185e-05  2.674e-03   0.012 0.990503    
cycleways  

dhl          dpd        fedex         ruch          ups log(tot + 1) 
    5.915307     2.091196     1.256635     2.593822     1.849121     2.974147 
   fem_ratio    buildings        shops        parks      forests      schools 
    1.577093     1.290085     1.618694     2.613386     1.132100     3.062693 
    railways    cycleways     parkings    crossings    bus_stops 
    1.323077     2.729662     2.212901     7.157296     2.471373


	studentized Breusch-Pagan test

data:  ols
BP = 111.49, df = 17, p-value = 6.313e-16



	RESET test

data:  ols
RESET = 7.3525, df1 = 17, df2 = 361, p-value = 1.501e-15



	Global Moran I for regression residuals

data:  
model: lm(formula = formula, data = df)
weights: df.cont.listw

Moran I statistic standard deviate = 0.86543, p-value = 0.1934
alternative hypothesis: greater
sample estimates:
Observed Moran I      Expectation         Variance 
    0.0119504900    -0.0105986646     0.0006788895 


In [45]:
intercept_only <- lm(inpost~1, data=df)
all <- lm(formula, data=df)
both <- step(intercept_only, direction='both', scope=formula(all), trace=0)

Based on Stepwise Regression we obtained final formula of the model and we are able to run our final OLS with diagnostic part. We see that nearly all variables in the model are significant (of course due to the heteroscedasticity we use biased error variance estimator - we omitted application of the robust matrix due to lack of time). There is no collinearity in the model, but again we did not pass Ramsey and B-P tests. What is more residuals from the model are not spatially dependent. In case of interpretation we can see that: logistic companies have positive sign as expected; the greater the total density the more InPost pick-up points are stated; fem_ratio results is interesting because its sign is negative (more woman less pick-up points), in addition, the negative relationship between parks and pickup points is interesting but intuitive. There is positive relationship between pickup points and schools, parkings, cycleways and bus stops. It is also intuitive!

In [46]:
formula = formula(both)
ols <- lm(formula, data=df)
summary(ols)
cat("AIC", AIC(ols))
car::vif(ols)
bptest(ols) 
resettest(ols, power=2, type="regressor")
lm.morantest(ols, df.cont.listw) 


Call:
lm(formula = formula, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.8521 -0.3876 -0.1289  0.2793  4.4299 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.257783   0.182849  -1.410  0.15940    
schools       0.122166   0.020100   6.078 2.93e-09 ***
dhl           0.171940   0.036511   4.709 3.48e-06 ***
parkings      0.012623   0.003094   4.080 5.48e-05 ***
cycleways     0.017109   0.004353   3.931  0.00010 ***
log(tot + 1)  0.173857   0.035023   4.964 1.04e-06 ***
parks        -0.193973   0.047366  -4.095 5.14e-05 ***
ups           0.361201   0.132367   2.729  0.00665 ** 
dpd           0.261543   0.115585   2.263  0.02421 *  
fem_ratio    -0.004469   0.001989  -2.247  0.02523 *  
bus_stops     0.036833   0.023741   1.551  0.12161    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.9197 on 385 degrees of freedom
Multiple R-squared:  0.7276,	Adjusted R-squared:  0.7206 
F-stati

AIC 1070.335

schools          dhl     parkings    cycleways log(tot + 1)        parks 
    2.665270     3.169044     1.908603     2.221025     2.811209     2.403163 
         ups          dpd    fem_ratio    bus_stops 
    1.666114     2.023179     1.555686     2.254750


	studentized Breusch-Pagan test

data:  ols
BP = 115.78, df = 10, p-value < 2.2e-16



	RESET test

data:  ols
RESET = 11.021, df1 = 10, df2 = 375, p-value < 2.2e-16



	Global Moran I for regression residuals

data:  
model: lm(formula = formula, data = df)
weights: df.cont.listw

Moran I statistic standard deviate = 0.71454, p-value = 0.2374
alternative hypothesis: greater
sample estimates:
Observed Moran I      Expectation         Variance 
    0.0113325967    -0.0074311675     0.0006895866 


### Spatial models with 3, 2, or 1 spatial components

In [47]:
GNS <- sacsarlm(formula, data=df, listw=df.cont.listw, type="sacmixed", method="LU")
summary(GNS)


Call:sacsarlm(formula = formula, data = df, listw = df.cont.listw, 
    type = "sacmixed", method = "LU")

Residuals:
      Min        1Q    Median        3Q       Max 
-3.754389 -0.361068 -0.076336  0.271624  3.595976 

Type: sacmixed 
Coefficients: (numerical Hessian approximate standard errors) 
                   Estimate Std. Error z value  Pr(>|z|)
(Intercept)      -0.1892880  0.4461054 -0.4243 0.6713380
schools           0.1208664  0.0227048  5.3234 1.019e-07
dhl               0.1962639  0.0445578  4.4047 1.059e-05
parkings          0.0059313  0.0036499  1.6251 0.1041487
cycleways         0.0164590  0.0049380  3.3331 0.0008587
log(tot + 1)      0.1349492  0.0378908  3.5615 0.0003687
parks            -0.1756511  0.0494775 -3.5501 0.0003851
ups               0.4202907  0.1360512  3.0892 0.0020069
dpd               0.2239553  0.1174352  1.9071 0.0565136
fem_ratio        -0.0037182  0.0019435 -1.9132 0.0557240
bus_stops         0.0285210  0.0258514  1.1033 0.2699121
lag.schools    

In [49]:
SAC <- sacsarlm(formula, data=df, listw=df.cont.listw)
summary(SAC)


Call:sacsarlm(formula = formula, data = df, listw = df.cont.listw)

Residuals:
      Min        1Q    Median        3Q       Max 
-3.742778 -0.378442 -0.082933  0.251057  4.336691 

Type: sac 
Coefficients: (asymptotic standard errors) 
               Estimate Std. Error z value  Pr(>|z|)
(Intercept)  -0.2285383  0.1732757 -1.3189  0.187193
schools       0.1173015  0.0192607  6.0902 1.128e-09
dhl           0.1519248  0.0356379  4.2630 2.017e-05
parkings      0.0117739  0.0029482  3.9935 6.510e-05
cycleways     0.0135092  0.0044227  3.0545  0.002254
log(tot + 1)  0.1480035  0.0352868  4.1943 2.737e-05
parks        -0.2093795  0.0460939 -4.5425 5.560e-06
ups           0.3964271  0.1306332  3.0347  0.002408
dpd           0.2521028  0.1130940  2.2291  0.025804
fem_ratio    -0.0042107  0.0019415 -2.1688  0.030098
bus_stops     0.0287761  0.0228751  1.2580  0.208405

Rho: 0.17813
Asymptotic standard error: 0.073257
    z-value: 2.4315, p-value: 0.015035
Lambda: -0.13362
Asymptotic standard 

In [50]:
SDM <- errorsarlm(formula, data=df, listw=df.cont.listw, etype="mixed")
summary(SDM)


Call:errorsarlm(formula = formula, data = df, listw = df.cont.listw, 
    etype = "mixed")

Residuals:
      Min        1Q    Median        3Q       Max 
-3.760312 -0.365729 -0.075627  0.271674  3.603924 

Type: error 
Coefficients: (asymptotic standard errors) 
                    Estimate  Std. Error z value  Pr(>|z|)
(Intercept)      -0.17645482  0.42142109 -0.4187 0.6754253
schools           0.12092515  0.02199218  5.4986 3.829e-08
dhl               0.19673895  0.04426460  4.4446 8.805e-06
parkings          0.00583431  0.00360056  1.6204 0.1051483
cycleways         0.01647383  0.00484529  3.4000 0.0006739
log(tot + 1)      0.13436113  0.03792020  3.5433 0.0003952
parks            -0.17550184  0.04948822 -3.5463 0.0003906
ups               0.41637263  0.13400568  3.1071 0.0018892
dpd               0.22459114  0.11692782  1.9208 0.0547610
fem_ratio        -0.00368692  0.00193419 -1.9062 0.0566261
bus_stops         0.02828919  0.02562449  1.1040 0.2695973
lag.schools       0.00050302

In [51]:
SDEM <- errorsarlm(formula, data=df, listw=df.cont.listw, etype="emixed")
summary(SDEM)


Call:errorsarlm(formula = formula, data = df, listw = df.cont.listw, 
    etype = "emixed")

Residuals:
      Min        1Q    Median        3Q       Max 
-3.760312 -0.365729 -0.075627  0.271674  3.603924 

Type: error 
Coefficients: (asymptotic standard errors) 
                    Estimate  Std. Error z value  Pr(>|z|)
(Intercept)      -0.17645482  0.42142109 -0.4187 0.6754253
schools           0.12092515  0.02199218  5.4986 3.829e-08
dhl               0.19673895  0.04426460  4.4446 8.805e-06
parkings          0.00583431  0.00360056  1.6204 0.1051483
cycleways         0.01647383  0.00484529  3.4000 0.0006739
log(tot + 1)      0.13436113  0.03792020  3.5433 0.0003952
parks            -0.17550184  0.04948822 -3.5463 0.0003906
ups               0.41637263  0.13400568  3.1071 0.0018892
dpd               0.22459114  0.11692782  1.9208 0.0547610
fem_ratio        -0.00368692  0.00193419 -1.9062 0.0566261
bus_stops         0.02828919  0.02562449  1.1040 0.2695973
lag.schools       0.0005030

In [52]:
SEM <- errorsarlm(formula, data=df, listw=df.cont.listw)
summary(SEM)


Call:errorsarlm(formula = formula, data = df, listw = df.cont.listw)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.86442 -0.38388 -0.13033  0.28413  4.43792 

Type: error 
Coefficients: (asymptotic standard errors) 
               Estimate Std. Error z value  Pr(>|z|)
(Intercept)  -0.2513907  0.1825858 -1.3768   0.16856
schools       0.1222839  0.0199770  6.1212 9.285e-10
dhl           0.1744698  0.0362854  4.8083 1.522e-06
parkings      0.0122151  0.0030927  3.9497 7.826e-05
cycleways     0.0171933  0.0043202  3.9797 6.899e-05
log(tot + 1)  0.1715968  0.0348729  4.9206 8.627e-07
parks        -0.1926390  0.0468210 -4.1144 3.882e-05
ups           0.3558354  0.1300344  2.7365   0.00621
dpd           0.2611246  0.1137346  2.2959   0.02168
fem_ratio    -0.0043836  0.0019600 -2.2365   0.02532
bus_stops     0.0363045  0.0236105  1.5376   0.12414

Lambda: 0.051161, LR test value: 0.23035, p-value: 0.63126
Asymptotic standard error: 0.09221
    z-value: 0.55483, p-value: 0.5790

In [53]:
SAR <- lagsarlm(formula, data=df, listw=df.cont.listw)
summary(SAR)


Call:lagsarlm(formula = formula, data = df, listw = df.cont.listw)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.80128 -0.37166 -0.10260  0.24129  4.37771 

Type: lag 
Coefficients: (asymptotic standard errors) 
               Estimate Std. Error z value  Pr(>|z|)
(Intercept)  -0.2341478  0.1790217 -1.3079 0.1908972
schools       0.1189272  0.0197130  6.0329 1.610e-09
dhl           0.1613324  0.0362029  4.4563 8.337e-06
parkings      0.0113351  0.0030435  3.7244 0.0001958
cycleways     0.0145720  0.0044047  3.3083 0.0009386
log(tot + 1)  0.1515994  0.0352620  4.2992 1.714e-05
parks        -0.2043753  0.0465837 -4.3873 1.148e-05
ups           0.3763571  0.1298082  2.8993 0.0037396
dpd           0.2542829  0.1130757  2.2488 0.0245262
fem_ratio    -0.0041514  0.0019474 -2.1318 0.0330254
bus_stops     0.0302281  0.0233637  1.2938 0.1957335

Rho: 0.13838, LR test value: 5.3541, p-value: 0.020673
Asymptotic standard error: 0.057539
    z-value: 2.4049, p-value: 0.016177
Wald 

In [54]:
SLX <- lmSLX(formula, data=df, listw=df.cont.listw)
summary(SLX)


Call:
lm(formula = formula(paste("y ~ ", paste(colnames(x)[-1], collapse = "+"))), 
    data = as.data.frame(x), weights = weights)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7586 -0.3556 -0.0735  0.2721  3.6347 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)      -0.176261   0.418246  -0.421 0.673684    
schools           0.120789   0.022690   5.323 1.76e-07 ***
dhl               0.195098   0.045651   4.274 2.44e-05 ***
parkings          0.005944   0.003725   1.596 0.111351    
cycleways         0.016424   0.004999   3.285 0.001115 ** 
log.tot...1.      0.135231   0.039192   3.450 0.000623 ***
parks            -0.176600   0.050968  -3.465 0.000592 ***
ups               0.416330   0.137141   3.036 0.002567 ** 
dpd               0.225329   0.119836   1.880 0.060841 .  
fem_ratio        -0.003700   0.001987  -1.862 0.063436 .  
bus_stops         0.028342   0.026456   1.071 0.284729    
lag.schools       0.003063   0.056190   0.055 0.95

Again SAR model seems to be the best in case of spatial terms significance and number of relevant variables! 
For Cracow BIC and AIC results also indicates SAR as the best model!

In [55]:
cat("ols",BIC(ols) ,'\n')
cat("GNS",BIC(GNS),'\n')
cat("SAC",BIC(SAC),'\n')
cat("SDM",BIC(SDM),'\n')
cat("SDEM",BIC(SDEM),'\n')
cat("SAR",BIC(SAR),'\n')
cat("SLX",BIC(SLX),'\n')
cat("SEM",BIC(SEM),'\n')
cat('\n')
cat("ols",AIC(ols) ,'\n')
cat("GNS",AIC(GNS),'\n')
cat("SAC",AIC(SAC),'\n')
cat("SDM",AIC(SDM),'\n')
cat("SDEM",AIC(SDEM),'\n')
cat("SAR",AIC(SAR),'\n')
cat("SLX",AIC(SLX),'\n')
cat("SEM",AIC(SEM),'\n')

ols 1118.112 
GNS 1171.356 
SAC 1123.65 
SDM 1165.455 
SDEM 1165.455 
SAR 1118.74 
SLX 1159.766 
SEM 1123.863 

ols 1070.335 
GNS 1075.803 
SAC 1067.91 
SDM 1073.883 
SDEM 1073.883 
SAR 1066.981 
SLX 1072.175 
SEM 1072.105 


Now we looked for better formula for SAR and we found it in the general to specific procedure.

In [67]:
formula_max = inpost ~ dhl + dpd + fedex + ruch + ups +
            log(tot + 1) + fem_ratio + 
            buildings + shops + parks + forests + schools + 
            railways + cycleways + parkings + crossings + bus_stops

formula_gen_to_specifc = inpost ~ dhl + dpd + ruch + ups +
            log(tot + 1) + fem_ratio + 
            parks + schools + 
            cycleways + parkings

SAR_g2s<- lagsarlm(formula_gen_to_specifc, data=df, listw=df.cont.listw)
summary(SAR_g2s) #the same as previously!


Call:
lagsarlm(formula = formula_gen_to_specifc, data = df, listw = df.cont.listw)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.71682 -0.36660 -0.09821  0.25738  4.45771 

Type: lag 
Coefficients: (asymptotic standard errors) 
               Estimate Std. Error z value  Pr(>|z|)
(Intercept)  -0.2358848  0.1789241 -1.3184 0.1873862
dhl           0.1474649  0.0390720  3.7742 0.0001605
dpd           0.2694286  0.1125832  2.3932 0.0167044
ruch          0.1777519  0.1225243  1.4507 0.1468500
ups           0.3238056  0.1339810  2.4168 0.0156575
log(tot + 1)  0.1531382  0.0352776  4.3409 1.419e-05
fem_ratio    -0.0041838  0.0019456 -2.1504 0.0315219
parks        -0.2068055  0.0466168 -4.4363 9.153e-06
schools       0.1132504  0.0203474  5.5658 2.609e-08
cycleways     0.0160661  0.0042884  3.7464 0.0001794
parkings      0.0124737  0.0030467  4.0941 4.238e-05

Rho: 0.14868, LR test value: 6.2791, p-value: 0.012217
Asymptotic standard error: 0.057206
    z-value: 2.599, p-value:

### Direct and indirect impacts

Now we move to the statistical inference part.

Let's start with logistic companies variables. We see that either in case of direct and indirect impacts all parameters for logistic companies (apart from Ruch total, and UPS indirect) are significant (at least around 10%). Internalization effects are greater than spill-over, but still spill-over effects are significant. In case of demographic variables we claim that total population density is important variable for the model in case of direct and indirect impact (has positive sign). Feminization ratio in case of direct impact is also significant and has negative sign (it is quite non intuitive). In case of point of interest data we see that number of schools, number of parks, number of cycleways and number of parkings are significant in the model with respectively positive, negative positive, and positive sign. We claim that it is intuitive.

In [71]:
# distribution of total impact 
# vector of traces of powers of a spatial weights matrix
# converting censored listw to CsparseMatrix form
W.c <- as(as_dgRMatrix_listw(df.cont.listw), "CsparseMatrix") 
# the default values for the number of powers is 30
trMat<-trW(W.c, type="mult") 

SAR_imp<-impacts(SAR_g2s, tr=trMat, R=200)
summary(SAR_imp, zstats=TRUE, short=TRUE)

Impact measures (lag, trace):
                   Direct      Indirect        Total
dhl           0.147953001  0.0252655979  0.173218599
dpd           0.270320307  0.0461619849  0.316482292
ruch          0.178340242  0.0304547581  0.208795000
ups           0.324877326  0.0554785630  0.380355889
log(tot + 1)  0.153644998  0.0262376074  0.179882606
fem_ratio    -0.004197681 -0.0007168284 -0.004914509
parks        -0.207489962 -0.0354325896 -0.242922552
schools       0.113625255  0.0194035267  0.133028781
cycleways     0.016119260  0.0027526493  0.018871909
parkings      0.012514984  0.0021371554  0.014652139
Simulation results ( variance matrix):
Simulated standard errors
                  Direct     Indirect       Total
dhl          0.041731285 0.0130662719 0.048890019
dpd          0.110718063 0.0287868162 0.130905207
ruch         0.128213530 0.0284477403 0.151676217
ups          0.139900865 0.0365312845 0.167022198
log(tot + 1) 0.033852240 0.0123982027 0.039850455
fem_ratio    0.0019854

## Conclusions from spatial dependence models

We combine our statistical inference for Warsaw and Cracow. Base of above research we can generally claim that InPost has deployed parcel pick-up points in line with the competitors - as the number of InPost pick-up points increases, the number of competitors' pick-up points increases (ignoring competitors not significant in the regression). We demonstrate that our control variables from the demographic group and points of interest are also significant, so InPost follows modeling logic and literature guidance for instance such variables are positively related with number of InPost's pick-up points: number of cycleways, number of parkings, number of schools, number of crossings. As it turned out a significant spatial effect in multivariate econometric models is Spatial Lag. It is quite intuitive that spillover effect exists for this problem.